In [1]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import codecs
import time
from SentenceSim import sentence_sim

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [2]:
cluster_num = 700
data_size = 5000
algorithm = "past"
past_threshold = 0.90

In [ ]:
start_time = time.time()
filename = "../../Data/1to100_chat_norm3.txt" #input("filename: ")
with open(filename, 'r', encoding='utf-8-sig') as data_file:
    chats = data_file.readlines()
    chats = list(map(lambda chat: chat.strip(), chats)) # to delete newline character
print(len(chats))
print("time: %.3f secs" % (time.time() - start_time))

In [ ]:
start_time = time.time()
sim_matrix = sentence_sim(chats[:data_size])
print("time: %.3f secs" % (time.time() - start_time))

In [ ]:
start_time = time.time()

if algorithm == "hc":
    clusterer = AgglomerativeClustering(n_clusters=cluster_num, affinity='precomputed', linkage='average').fit(1 - sim_matrix)
    clusters = [[] for _ in range(cluster_num)]
    label_list = clusterer.labels_
    for idx in range(len(label_list)):
        clusters[label_list[idx]].append(idx)
elif algorithm == "past":
    clusters = []
    idx_x, idx_y = np.where(sim_matrix > past_threshold)
    for idx in range(len(idx_x)):
        x, y = idx_x[idx], idx_y[idx]
        if y <= x:
            continue
        cluster_exist = False
        for cluster_idx in range(len(clusters)):
            if x in clusters[cluster_idx]:
                cluster_exist = True
                clusters[cluster_idx].add(y)
                break
        if cluster_exist == False:
            cluster = {x, y}
            clusters.append(cluster)

cluster_lens = list()
for idx in range(len(clusters)):
    cluster_lens.append((len(clusters[idx]), idx))
cluster_lens.sort(reverse=True)
print("time: %.3f secs" % (time.time() - start_time))

In [ ]:
with codecs.open(f"{algorithm}_{len(clusters)}_{data_size}({chunk_num}).txt", 'w', encoding='utf-8') as output_file:
    for len, idx in cluster_lens:
        if len == 1:
            continue
        output_file.write("---------------------------------------\n")
        for chat in clusters[idx]:
            output_file.write(chats[chat] + '\n')

In [ ]:
similarity_threshold = 0.92

start_time = time.time()
close_chats = np.where(sim_matrix > similarity_threshold)
relate_counts = [[0, idx] for idx in range(data_size)]
for idx in range(len(close_chats[0])):
    relate_counts[close_chats[0][idx]][0] += 1
relate_counts.sort(reverse=True)
print(relate_counts[:30])
print("time: %.3f secs" % (time.time() - start_time))

In [ ]:
for idx in range(30):
    print(chats[relate_counts[idx][1]])